In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import regex as re
import statsmodels.api as sm
import statsmodels.formula.api as smf

## PSET 1

In [2]:
nba = pd.read_csv("nba.csv")
nba#.head()

married    wage  exper  age  coll  games  minutes  guard  forward  \
0          1  1002.5      4   27     4     77     2867      1        0   
1          1  2030.0      5   28     4     78     2789      1        0   
2          0   650.0      1   25     4     74     1149      0        0   
3          0  2030.0      5   28     4     47     1178      0        1   
4          0   755.0      3   24     4     82     2096      1        0   
..       ...     ...    ...  ...   ...    ...      ...    ...      ...   
264        1  3210.0      7   29     4     79     2638      1        0   
265        1   715.0      5   31     4     75     1084      0        1   
266        1   600.0     11   33     3     67     1197      1        0   
267        0  2500.0      6   28     4     78     2113      0        0   
268        0  2000.0     12   33     3     30      282      0        1   

     center  points  rebounds  assists  draft  allstar  avgmin  black  \
0         0      16         4        5   19.0        0   37.23      1   
1         0      13         3        9   28.0        0   35.76      1   
2         1       6         3        0   19.0        0   15.53      1   
3         0       7         5        2    1.0        0   25.06      1   
4         0      11         4        3   24.0        0   25.56      1   
..      ...     ...       ...      ...    ...      ...     ...    ...   
264       0      20         3        3   11.0        1   33.39      1   
265       0       5         3        1   54.0        0   14.45      1   
266       0      10         2        2    4.0        0   17.87      1   
267       1      16         6        2    2.0        0   27.09      0   
268       0       2         3        1    5.0        0    9.40      1   

     children  
0           0  
1           1  
2           0  
3           0  
4           0  
..        ...  
264         0  
265         1  
266         1  
267         0  
268         0  

[269 rows x 18 columns]

In [59]:
def ttest(string, df_name = 'df'):
    print("import numpy as np")
    print("from scipy import stats")
    eq_var = True
    nan_pol = 'propagate'

    string = string.replace("ttest ", "")
    catvar_split = string.split("(")
    catvar = catvar_split[-1].split(")")[0]
    if catvar_split[-1].split(")")[1].strip() == 'unequal':
        eq_var = False
    catvar_split[0] = "(".join(catvar_split[:-1])
    var = catvar_split[0].split(",")[0]
    if 'if !missing' in var:
        var = var.split(" ")[0]
        nan_pol = 'omit'    
    print("### First, we must filter the DataFrame to obtain the right values")
    print(f"catvar_vals = np.unique({df_name}['{catvar}'])")
    print(f"df_1 = {df_name}[{df_name}['{catvar}'] == catvar_vals[0]]")
    print(f"df_2 = {df_name}[{df_name}['{catvar}'] == catvar_vals[1]]")
    print("### Then, we can run our t-test")
    print(f"stats.ttest_ind(df_1['{var}'], df_2['{var}'], equal_var={eq_var}, nan_policy='{nan_pol}')")

In [60]:
ttest("ttest wage, by(guard)","nba")

import numpy as np
from scipy import stats
### First, we must filter the DataFrame to obtain the right values
catvar_vals = np.unique(nba['guard'])
df_1 = nba[nba['guard'] == catvar_vals[0]]
df_2 = nba[nba['guard'] == catvar_vals[1]]
### Then, we can run our t-test
stats.ttest_ind(df_1['wage'], df_2['wage'], equal_var=True, nan_policy='propagate')


In [61]:
ttest("ttest wage, by(guard) unequal", "nba")

import numpy as np
from scipy import stats
### First, we must filter the DataFrame to obtain the right values
catvar_vals = np.unique(nba['guard'])
df_1 = nba[nba['guard'] == catvar_vals[0]]
df_2 = nba[nba['guard'] == catvar_vals[1]]
### Then, we can run our t-test
stats.ttest_ind(df_1['wage'], df_2['wage'], equal_var=False, nan_policy='propagate')


In [6]:
# Q1c, 1e

def filter_gen(df_name, string):
    if string.startswith("gen"):
        string = string.replace("gen ", "")
        str_split = string.split("=",1)
        str_split = [x.strip() for x in str_split]
        new_col_name = str_split[0]
        if str_split[1].count("=")>0:
            if str_split[1].startswith("("):
                str_split[1] = str_split[1][1:-1]
            filter_split = str_split[1].split(" ", 1)
            new_string = f"{df_name}['{new_col_name}'] = {df_name}['{filter_split[0]}'] {filter_split[1]}"
            new_string_split = new_string.split(" = ", 1)
            print(new_string)
        else:
            words_lst = re.findall(r'\w\w+',str_split[1])
            for word in words_lst:
                if word not in ['log']:
                    str_split[1] = str_split[1].replace(word, f"{df_name}['{word}']")
                else:
                    str_split[1] = str_split[1].replace(word, f"np.{word}")
            new_string = f"{df_name}['{new_col_name}'] = {str_split[1]}"
            print(new_string)
            

In [7]:
filter_gen("nba","gen degree = (coll >= 4)")

nba['degree'] = nba['coll'] >= 4


In [8]:
filter_gen("nba","gen productivity = points/(minutes/games)")

nba['productivity'] = nba['points']/(nba['minutes']/nba['games'])


In [9]:
# Q1g

def corr(df_name, string):
    if string.startswith("corr"):
        string = string.replace("corr ", "")
        words_lst = re.findall(r'[a-zA-Z]+',string)
        print(f"{df_name}[{words_lst}].corr()")

In [10]:
corr("nba","corr points assists rebounds")

nba[['points', 'assists', 'rebounds']].corr()


In [11]:
filter_gen("nba","gen index = points + rebounds + 2*assists")

nba['index'] = nba['points'] + nba['rebounds'] + 2*nba['assists']


In [12]:
filter_gen("nba","gen payoff = index/wage")

nba['payoff'] = nba['index']/nba['wage']


In [13]:
pollution = pd.read_csv("pollution.csv")
pollution.head()

year       countryname countrycode           gdp      gdppc         co2  \
0  2010            Zambia         ZMB  9.799629e+09   741.4421    2427.554   
1  2010  French Polynesia         PYF           NaN        NaN     883.747   
2  2010            Monaco         MCO           NaN        NaN         NaN   
3  2010           Ukraine         UKR  9.057726e+10  1974.6212  304804.720   
4  2010     Venezuela, RB         VEN  1.750000e+11  6010.0270  201747.340   

      co2pc  population  oecd  
0  0.183669    13216985   0.0  
1  3.296764      268065   0.0  
2       NaN       36845   0.0  
3  6.644867    45870700   0.0  
4  6.946437    29043283   0.0

In [14]:
ttest("ttest co2pc if !missing(countrycode), by(oecd) unequal", "pollution")

catvar_vals = np.unique(pollution['oecd'])
if len(catvar_vals) != 2:
    raise ValueError(f'The categorical variable (oecd) doesn\'t have 2 groups')
df_1 = pollution[pollution['oecd'] == catvar_vals[0]]
df_2 = pollution[pollution['oecd'] == catvar_vals[1]]
ttest = stats.ttest_ind(df_1['co2pc'], df_2['co2pc'], equal_var=False, nan_policy='omit')
t_stat = ttest.statistic
p_val = ttest.pvalue
print(f'T-stat: {t_stat}, P-value: {p_val}')


In [31]:
catvar_vals = np.unique(pollution['oecd'])
if len(catvar_vals) != 2:
    raise ValueError(f'The categorical variable (oecd) doesn\'t have 2 groups')
df_1 = pollution[pollution['oecd'] == catvar_vals[0]]
df_2 = pollution[pollution['oecd'] == catvar_vals[1]]
stats.ttest_ind(df_1['co2pc'], df_2['co2pc'], equal_var=False, nan_policy='omit')

TtestResult(statistic=-5.682465161974536, pvalue=2.683581293169389e-07, df=71.47718715993969)

In [15]:
filter_gen("pollution","gen log_gdp = log(gdp)")
filter_gen("pollution","gen log_co2 = log(co2)")

pollution['log_gdp'] = np.log(pollution['gdp'])
pollution['log_co2'] = np.log(pollution['co2'])


In [16]:
pollution['log_gdp'] = np.log(pollution['gdp'])
pollution['log_co2'] = np.log(pollution['co2'])

In [17]:
def scatter(string, df_name = 'df'):
    if string.startswith("twoway"):
        string = string.replace("twoway (","")[:-2]
        command = string.split(",")[0].strip()
        customizations = string.split(",")[1].strip()
        df = eval(df_name)
        if command.startswith("scatter"):
            var_1 = command.split(" ")[1]
            var_2 = command.split(" ")[2]
            print(f"plt.scatter({df_name}['{var_1}'], {df_name}['{var_2}']);")
        customizations_split = customizations.split(")")
        for word in customizations_split:
            word = word.strip()
            if word.startswith("xtitle"):
                xtitle = word.split("(")[1]
                if xtitle.startswith("'"):
                    xtitle = xtitle[1:]
                if xtitle.endswith("'"):
                    xtitle = xtitle[:-1]
                print(f"plt.xlabel('{xtitle}');")
            if word.startswith("ytitle"):
                ytitle = word.split("(")[1]
                if ytitle.startswith("'"):
                    ytitle = ytitle[1:]
                if ytitle.endswith("'"):
                    ytitle = ytitle[:-1]
                print(f"plt.ylabel('{ytitle}');")

In [18]:
scatter("twoway (scatter log_gdp log_co2, xtitle('log gdp') ytitle('log co2'))","pollution")

plt.scatter(pollution['log_gdp'], pollution['log_co2']);
plt.xlabel('log gdp');
plt.ylabel('log co2');


In [19]:
v = np.random.random_sample(100)
x = 10 + 20 * v
u = np.random.normal(0, 100, 100)
y = 30 + 5 * x + u
df_all = pd.DataFrame(data={'v':v, 'x':x,'u':u,'y':y})
df_all.head()

v          x           u           y
0  0.497907  19.958145  -34.342513   95.448213
1  0.643911  22.878215   83.974146  228.365219
2  0.597268  21.945360   50.444549  190.171347
3  0.863052  27.261030  202.860614  369.165766
4  0.027564  10.551286    9.351561   92.107991

In [20]:
scatter(df_name="df_all", string='twoway (scatter x y, xtitle(x) ytitle(y))')

plt.scatter(df_all['x'], df_all['y']);
plt.xlabel('x');
plt.ylabel('y');


In [21]:
def describe(string, df_name='df'):
    if string.startswith('describe'):
        print(f'{df_name}.describe()')

In [22]:
la = pd.read_csv("la.csv")
la.head()

hispanic  citizen  black   exp       wage  female  education
0         1        1      0  14.0   5.288462       1          9
1         0        1      0  14.7   8.461538       1         13
2         0        1      0  14.7  10.416667       1         13
3         0        1      0  14.0  21.634615       1         14
4         1        0      0  12.0   3.365385       1         12

In [23]:
describe("describe","la")

la.describe()


In [24]:
def hist(string, df_name='df'):
    if string.startswith('histogram'):
        string = string.replace('histogram ','')
        str_split = string.split(", ")
        num_bins_change = False
        num_bins = 0
        for word in str_split:
            if word.startswith("bin"):
                num_bins_change = True
                num_bins = int(word.split("(")[1].split(")")[0])
        if num_bins_change == False:
            print(f"{df_name}.hist(column='{str_split[0]}');")
        else:
            print(f"{df_name}.hist(column='{str_split[0]}',bins={num_bins});")

In [25]:
hist("histogram wage, bin(80)","la");

la.hist(column='wage',bins=80);


## PSET 2

In [26]:
v = np.random.random_sample(100)
x = 10 + 20 * v
u = np.random.normal(0, 100, 100)
y = 30 + 5 * x + u

In [54]:
def reg(string, df_name = 'df'):
    print("import statsmodels.api as sm")
    string = string[4:]
    clustering_vars = ''
    first_half, second_half = string, string
    if "," in string:
        first_half = string.split(",")[0]
        second_half = string.split(",")[1]
    if "if" in first_half:
        print("### We first filter the DataFrame")
        if_statement = first_half.split("if")[1].strip()
        first_half = first_half.split("if")[0].strip()
        if "=" in if_statement:
            splitted_if = if_statement.split("=")
            print(f"{df_name}_filtered = {df_name}[{df_name}['{splitted_if[0].strip()}'] == {splitted_if[1].strip()}]")
        elif "<" in if_statement:
            splitted_if = if_statement.split("<")
            print(f"{df_name}_filtered = {df_name}[{df_name}['{splitted_if[0].strip()}'] < {splitted_if[1].strip()}]")
        elif ">" in if_statement:
            splitted_if = if_statement.split(">")
            print(f"{df_name}_filtered = {df_name}[{df_name}['{splitted_if[0].strip()}'] > {splitted_if[1].strip()}]")
        else:
            raise ValueError(f'Could not interpret if statement')
        df_name = df_name + "_filtered"
    y_var = first_half.split(" ")[0]
    x_var = "'" + "', '".join(first_half.split(" ")[1:]) + "'"
    if "vce" in second_half and "cluster" in second_half:
        print("### We first perform some string manipulations to ensure we are appropripately accounting for strings. This section seems complicated as this code is meant to be a general-purpose code for converting the Stata commands to Python. You can greatly simplify this step if you know exactly which variables you are interested in; simply extract those variables from the DataFrame directly. ")
        print(f"y_var = \"{y_var}\"")
        second_half_split = second_half.split("cluster")
        clustering_vars = second_half_split[1][:-1].strip().split(" ")
        print(f"clustering_vars = {clustering_vars}")
    if ' ' not in x_var:
        if clustering_vars:
            x_var_temp = x_var + ", '" + "', '".join(clustering_vars) + "'"
            print("### Dropping NaN/missing values")
            print(f"{df_name}_no_na = {df_name}[[{x_var_temp},'{y_var}']].dropna()")
        else:
            print("### Dropping NaN/missing values")
            print(f"{df_name}_no_na = {df_name}[[{x_var},'{y_var}']].dropna()")
        df_name = df_name + "_no_na"
        print("### Below, we extract the relevant variables from the DataFrame")
        print(f"x_df = {df_name}[{x_var}]")
    elif "i." in x_var:
        #raise ValueError(f'Indicators are currently not supported')
        print(f"{df_name}_with_dummies = {df_name}.copy()")
        df_name = df_name + "_with_dummies"
        print(f"x_var = \"{x_var}\"")
        x_var_new = "'" + "', '".join([i.strip("'") for i in x_var.split(", ") if not i.strip("'").startswith("i.")]) + "'"
        print("x_var_new = \"'\" + \"', '\".join([i.strip(\"'\") for i in x_var.split(\", \") if not i.strip(\"'\").startswith(\"i.\")]) + \"'\"")
        print("### The below section adds in relevant indicator variables, ensuring they have interpretable names")
        print(f"indicator_list = [m.strip(\"'\") for m in x_var.split('i.')[1:]]")
        print("for indicator in indicator_list:")
        print(f"    dummies = pd.get_dummies({df_name}[indicator])")
        print(f"    dummies = dummies.iloc[:,1:]")
        print(f"    dummies.columns = [str(x) + '_' + str(indicator) for x in dummies.columns]")
        print(f"    {df_name} = pd.concat([{df_name},dummies],axis=1)")
        print("    x_var_new = x_var_new + \", '\" + \"', '\".join(dummies.columns) + \"'\"")
        print("x_var = x_var_new")
        print("### This helps ensure the clustered variables are extracted from the DataFrame")
        print("if clustering_vars:")
        print("    x_var_temp = x_var + \", '\" + \"', '\".join(clustering_vars) + \"'\"")
        if clustering_vars:
            x_var_temp = x_var + ", '" + "', '".join(clustering_vars) + "'"
            print(f"var_list = x_var_temp +\", '\"+ str(y_var) + \"'\"")
            print("### Dropping NaN/missing values")
            print(f"{df_name}_no_na = {df_name}[[i.strip(\"'\") for i in var_list.split(\", \")]].dropna()")
        else:
            print("### Dropping NaN/missing values")
            print(f"{df_name}_no_na = {df_name}[[{x_var},'{y_var}']].dropna()")
        df_name = df_name + "_no_na"
        print("### Below, we extract the relevant variables from the DataFrame")
        print(f"x_df = {df_name}[[i.strip(\"'\") for i in x_var.split(\", \")]]")
    else:
        if clustering_vars:
            x_var_temp = x_var + ", '" + "', '".join(clustering_vars) + "'"
            print("### Dropping NaN/missing values")
            print(f"{df_name}_no_na = {df_name}[[{x_var_temp},'{y_var}']].dropna()")
        else:
            print("### Dropping NaN/missing values")
            print(f"{df_name}_no_na = {df_name}[[{x_var},'{y_var}']].dropna()")
        df_name = df_name + "_no_na"
        print("### Below, we extract the relevant variables from the DataFrame")
        print(f"x_df = {df_name}[[{x_var}]]")
    print(f"y_df = {df_name}['{y_var}']")
    print("### We now define the model, fit it to the data and then view a summary of the results")
    if 'noconstant' in second_half:
        print("model = sm.OLS(y_df, x_df)")
    else:
        print("model = sm.OLS(y_df, sm.add_constant(x_df))")
    if "vce" not in second_half:
        print("result = model.fit()")
    elif "vce" in second_half and "robust" in second_half:
        print("result = model.fit(cov_type='HC1')")
    elif "vce" in second_half and "cluster" in second_half:
        print(f"result = model.fit(cov_type='cluster', cov_kwds={{'groups': {df_name}{clustering_vars}}})")
    else:
        raise ValueError(f'VCE type not recognized')
    print("result.summary()")

In [55]:
reg("reg wage exp", "la")

import statsmodels.api as sm
### Dropping NaN/missing values
la_no_na = la[['exp','wage']].dropna()
### Below, we extract the relevant variables from the DataFrame
x_df = la_no_na['exp']
y_df = la_no_na['wage']
### We now define the model, fit it to the data and then view a summary of the results
model = sm.OLS(y_df, sm.add_constant(x_df))
result = model.fit()
result.summary()


In [58]:
import statsmodels.api as sm
### Dropping NaN/missing values
la_no_na = la[['exp','wage']].dropna()
### Below, we extract the relevant variables from the DataFrame
x_df = la_no_na['exp']
y_df = la_no_na['wage']
### We now define the model, fit it to the data and then view a summary of the results
model = sm.OLS(y_df, sm.add_constant(x_df))
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.4714
Date:                Sat, 03 Feb 2024   Prob (F-statistic):              0.493
Time:                        12:18:22   Log-Likelihood:                -3541.0
No. Observations:                 863   AIC:                             7086.
Df Residuals:                     861   BIC:                             7096.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.0816      3.917      4.105      0.000       8.393      23.771
exp           -0.2135      0.311     -0.687      0.493      -0.824       0.397
==============================================================================
Omnibus:                     1263.062   Durbin-Watson:                   1.830
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           413853.334
Skew:                           8.176   Prob(JB):                         0.00
Kurtosis:                     109.028   Cond. No.                         99.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [56]:
reg("reg wage exp i.female, vce(cluster education)", "la")

import statsmodels.api as sm
### We first perform some string manipulations to ensure we are appropripately accounting for strings. This section seems complicated as this code is meant to be a general-purpose code for converting the Stata commands to Python. You can greatly simplify this step if you know exactly which variables you are interested in; simply extract those variables from the DataFrame directly. 
y_var = "wage"
clustering_vars = ['education']
la_with_dummies = la.copy()
x_var = "'exp', 'i.female'"
x_var_new = "'" + "', '".join([i.strip("'") for i in x_var.split(", ") if not i.strip("'").startswith("i.")]) + "'"
### The below section adds in relevant indicator variables, ensuring they have interpretable names
indicator_list = [m.strip("'") for m in x_var.split('i.')[1:]]
for indicator in indicator_list:
    dummies = pd.get_dummies(la_with_dummies[indicator])
    dummies = dummies.iloc[:,1:]
    dummies.columns = [str(x) + '_' + str(indicator) for x in dummies.columns]
   

In [57]:
import statsmodels.api as sm
### We first perform some string manipulations to ensure we are appropripately accounting for strings. This section seems complicated as this code is meant to be a general-purpose code for converting the Stata commands to Python. You can greatly simplify this step if you know exactly which variables you are interested in; simply extract those variables from the DataFrame directly. 
y_var = "wage"
clustering_vars = ['education']
la_with_dummies = la.copy()
x_var = "'exp', 'i.female'"
x_var_new = "'" + "', '".join([i.strip("'") for i in x_var.split(", ") if not i.strip("'").startswith("i.")]) + "'"
### The below section adds in relevant indicator variables, ensuring they have interpretable names
indicator_list = [m.strip("'") for m in x_var.split('i.')[1:]]
for indicator in indicator_list:
    dummies = pd.get_dummies(la_with_dummies[indicator])
    dummies = dummies.iloc[:,1:]
    dummies.columns = [str(x) + '_' + str(indicator) for x in dummies.columns]
    la_with_dummies = pd.concat([la_with_dummies,dummies],axis=1)
    x_var_new = x_var_new + ", '" + "', '".join(dummies.columns) + "'"
x_var = x_var_new
### This helps ensure the clustered variables are extracted from the DataFrame
if clustering_vars:
    x_var_temp = x_var + ", '" + "', '".join(clustering_vars) + "'"
var_list = x_var_temp +", '"+ str(y_var) + "'"
### Dropping NaN/missing values
la_with_dummies_no_na = la_with_dummies[[i.strip("'") for i in var_list.split(", ")]].dropna()
### Below, we extract the relevant variables from the DataFrame
x_df = la_with_dummies_no_na[[i.strip("'") for i in x_var.split(", ")]]
y_df = la_with_dummies_no_na['wage']
### We now define the model, fit it to the data and then view a summary of the results
model = sm.OLS(y_df, sm.add_constant(x_df))
result = model.fit(cov_type='cluster', cov_kwds={'groups': la_with_dummies_no_na['education']})
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.993
Date:                Sat, 03 Feb 2024   Prob (F-statistic):              0.183
Time:                        12:18:13   Log-Likelihood:                -3538.0
No. Observations:                 863   AIC:                             7082.
Df Residuals:                     860   BIC:                             7096.
Df Model:                           2                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         17.4218      3.698      4.712      0.000      10.175      24.669
exp           -0.2373      0.206     -1.153      0.249      -0.640       0.166
1_female      -2.4604      1.261     -1.951      0.051      -4.932       0.011
==============================================================================
Omnibus:                     1260.198   Durbin-Watson:                   1.842
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           409372.994
Skew:                           8.142   Prob(JB):                         0.00
Kurtosis:                     108.449   Cond. No.                         101.
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [30]:
y_var = "wage"
clustering_vars = ['education']
la_with_dummies = la.copy()
x_var = "'exp', 'i.female'"
x_var_new = "'" + "', '".join([i.strip("'") for i in x_var.split(", ") if not i.strip("'").startswith("i.")]) + "'"
indicator_list = [m.strip("'") for m in x_var.split('i.')[1:]]
for indicator in indicator_list:
    dummies = pd.get_dummies(la_with_dummies[indicator])
    dummies = dummies.iloc[:,1:]
    dummies.columns = [str(x) + '_' + str(indicator) for x in dummies.columns]
    la_with_dummies = pd.concat([la_with_dummies,dummies],axis=1)
    x_var_new = x_var_new + ", '" + "', '".join(dummies.columns) + "'"
x_var = x_var_new
if clustering_vars:
    x_var_temp = x_var + ", '" + "', '".join(clustering_vars) + "'"
var_list = x_var_temp +", '"+ str(y_var) + "'"
la_with_dummies_no_na = la_with_dummies[[i.strip("'") for i in var_list.split(", ")]].dropna()
x_df = la_with_dummies_no_na[[i.strip("'") for i in x_var.split(", ")]]
y_df = la_with_dummies_no_na['wage']
model = sm.OLS(y_df, sm.add_constant(x_df))
result = model.fit(cov_type='cluster', cov_kwds={'groups': la_with_dummies_no_na['education']})
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.993
Date:                Sat, 03 Feb 2024   Prob (F-statistic):              0.183
Time:                        11:51:19   Log-Likelihood:                -3538.0
No. Observations:                 863   AIC:                             7082.
Df Residuals:                     860   BIC:                             7096.
Df Model:                           2                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         17.4218      3.698      4.712      0.000      10.175      24.669
exp           -0.2373      0.206     -1.153      0.249      -0.640       0.166
1_female      -2.4604      1.261     -1.951      0.051      -4.932       0.011
==============================================================================
Omnibus:                     1260.198   Durbin-Watson:                   1.842
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           409372.994
Skew:                           8.142   Prob(JB):                         0.00
Kurtosis:                     108.449   Cond. No.                         101.
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""